In [ ]:
mortality_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/mortality_data_2000-2019.csv'
population_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/Population_par_commune.xlsx'

In [7]:
import pandas as pd

#################                weekly mortality data divided by municipality (2000-2019)
week_mor_mun = pd.read_csv(mortality_url)
week_mor_mun['WEEK'] = week_mor_mun['YEAR_WEEK'].str.slice(start=6).astype('int') # get week number
week_mor_mun.rename(columns = {'N_MASK':'N_DEATHS'}, inplace = True)
del week_mor_mun['YEAR_WEEK']
week_mor_mun = week_mor_mun[week_mor_mun.COD != 'external'] # do not need data for external causes of death
week_mor_mun = week_mor_mun[week_mor_mun.ARRON != 58000] # data for Arrondissement de La Louvière is only available for 2019 and therefore has been removed from dataset
print(week_mor_mun)
week_mor_mun.info()

       YEAR      COD  ARRON  N_DEATHS  WEEK
17     2000  natural  11000      31.0     1
18     2000  natural  12000      10.0     1
19     2000  natural  13000      14.0     1
20     2000  natural  21000      43.0     1
21     2000  natural  23000      14.0     1
...     ...      ...    ...       ...   ...
82579  2019  natural  84000       6.0    53
82580  2019  natural  85000       5.0    53
82581  2019  natural  91000      10.0    53
82582  2019  natural  92000      19.0    53
82583  2019  natural  93000       7.0    53

[45555 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 45555 entries, 17 to 82583
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   YEAR      45555 non-null  int64  
 1   COD       45555 non-null  object 
 2   ARRON     45555 non-null  int64  
 3   N_DEATHS  45210 non-null  float64
 4   WEEK      45555 non-null  int32  
dtypes: float64(1), int32(1), int64(2), object(1)
memory usage: 

In [147]:
##################               combine data sets from different excel sheets

# data with yearly population data divided by municipality
xls = pd. ExcelFile(population_url)

# create base dataframe
cols = ['municipality_code', 'municipality_name', 'male', 'female', 'total']
year_pop_mun = pd.DataFrame(columns=cols)
years = []
for i in range(21): # need poulation data for 21 years (years on both ends)
    df = pd.read_excel(xls, 'Population en '+str(2000+i), header=None, skiprows=4, nrows=646) # 2000-2019
    df.columns = cols
    length_of_df = df.loc[df['municipality_name'] == 'Viroinval'].index.tolist()[0] + 1 # get the last line of table
    years = years+[2000+i]*length_of_df
    year_pop_mun = pd.concat([year_pop_mun, df[:length_of_df]] , axis=0, ignore_index=True)
year_pop_mun['year'] = years
    
# # last dataframe is missing 8 municipalities
# year_pop_mun.drop(year_pop_mun.tail(4).index,inplace=True)
# year_pop_mun['year'] = years[:-8]

# convert population columns to int
convert_dict = {'male': int, 'female': int, 'total': int, 'municipality_code':int}
year_pop_mun = year_pop_mun.astype(convert_dict)

In [150]:
##################                assign population to weekly death rate data (at week 1)
existing_arrons = week_mor_mun['ARRON'].value_counts().index
week_mor_mun['population_t'] = np.nan
exceptions = []
for year in range(21):
    for arron in existing_arrons:
        value = year_pop_mun[(year_pop_mun['year'] == (2000+year)) & (year_pop_mun['municipality_code'] == arron)]['total'].tolist()
        if len(value) == 1: # check if population value exists in year_pop_mun
            condition = (week_mor_mun['ARRON'] == arron) & (week_mor_mun['WEEK'] == 1) & (week_mor_mun['YEAR'] == (2000+year))
            if (condition).any(): # check if a row with week 1 exists in week_mor_mun
                week_mor_mun.loc[condition, 'population_t'] = value[0]
            else:
                dummy_df = pd.DataFrame({'YEAR': [2000+year], 'COD': ['natural'], 'ARRON': [arron], 'N_DEATHS': [0], 'WEEK':[1], 'population_t':  [value[0]]})
                week_mor_mun = pd.concat([week_mor_mun, dummy_df] , axis=0, ignore_index=True)
        else:
            exceptions.append((year, arron))

In [151]:
exceptions

[(19, 54000), (20, 54000)]

In [8]:
week_mor_mun['ARRON'].value_counts()

11000    1061
44000    1061
73000    1061
72000    1061
71000    1061
63000    1061
62000    1061
61000    1061
57000    1061
56000    1061
55000    1061
53000    1061
52000    1061
12000    1061
46000    1061
45000    1061
91000    1061
41000    1061
38000    1061
92000    1061
36000    1061
35000    1061
34000    1061
33000    1061
42000    1061
31000    1061
25000    1061
24000    1061
23000    1061
21000    1061
13000    1061
83000    1060
85000    1060
93000    1060
84000    1060
51000    1060
43000    1060
37000    1060
32000    1060
64000    1060
81000    1059
82000    1058
54000    1007
Name: ARRON, dtype: int64

In [160]:
# get number of weeks for each year
no_weeks = [week_mor_mun[(week_mor_mun['YEAR'] == (2000+i))]['WEEK'].max() for i in range(20)] # weeks for each year
for year in range(20): # for each year
    for arron in existing_arrons: # for each arron
        if arron == 54000 and (year == 18 or year == 19): # 54000 population not available for 2019
            continue
        condition = (week_mor_mun['ARRON'] == arron) & (week_mor_mun['WEEK'] == 1)
        prev_year_pop = week_mor_mun.loc[condition & (week_mor_mun['YEAR'] == (2000+year)), 'population_t'].tolist()[0]
        next_year_pop = week_mor_mun.loc[condition & (week_mor_mun['YEAR'] == (2000+year+1)), 'population_t'].tolist()[0]
        inter_values = np.linspace(prev_year_pop, next_year_pop, no_weeks[year], endpoint=False, dtype=int)[1:]
        for ix, k in enumerate(inter_values):
            week_mor_mun.loc[(week_mor_mun['ARRON'] == arron) & (week_mor_mun['YEAR'] == (2000+year)) & (week_mor_mun['WEEK'] == ix + 2),'population_t'] = k

In [156]:
print(arron, year)

54000 18


In [154]:
week_mor_mun.to_csv('out/weekly_mortality_with_linear_interpolated_population_data.csv', index=False)